In [1]:
import pandas as pd
db = pd.read_csv('nursery.csv')
db.describe()

,parents,has_nurs,form,children,housing,finance,social,health,class
count,12960,12960,12960,12960,12960,12960,12960,12960,12960
unique,3,5,4,4,3,2,3,3,5
top,usual,proper,complete,1,convenient,convenient,nonprob,recommended,not_recom
freq,4320,2592,3240,3240,4320,6480,4320,4320,4320


In [2]:
descriptive = db.iloc[:, 0:8].values
target = db.iloc[:, 8].values

In [3]:
from sklearn.preprocessing import LabelEncoder
labelEncoder = LabelEncoder()

descriptive[:, 0] = labelEncoder.fit_transform(descriptive[:, 0])
descriptive[:, 1] = labelEncoder.fit_transform(descriptive[:, 1])
descriptive[:, 2] = labelEncoder.fit_transform(descriptive[:, 2])
descriptive[:, 3] = labelEncoder.fit_transform(descriptive[:, 3])
descriptive[:, 4] = labelEncoder.fit_transform(descriptive[:, 4])
descriptive[:, 5] = labelEncoder.fit_transform(descriptive[:, 5])
descriptive[:, 6] = labelEncoder.fit_transform(descriptive[:, 6])
descriptive[:, 7] = labelEncoder.fit_transform(descriptive[:, 7])


In [4]:
descriptive

array([[2, 3, 0, ..., 0, 0, 2],
       [2, 3, 0, ..., 0, 0, 1],
       [2, 3, 0, ..., 0, 0, 0],
       ...,
       [0, 4, 2, ..., 1, 1, 2],
       [0, 4, 2, ..., 1, 1, 1],
       [0, 4, 2, ..., 1, 1, 0]], shape=(12960, 8), dtype=object)

In [5]:
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Can't use OneHotEncoder if there is a numeric feature (3), because it loses the distance between the values
column_transformer = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0, 1, 2, 4, 5, 6, 7])], remainder='passthrough')
descriptive = np.array(column_transformer.fit_transform(descriptive))

descriptive

array([[0.0, 0.0, 1.0, ..., 0.0, 1.0, 0],
       [0.0, 0.0, 1.0, ..., 1.0, 0.0, 0],
       [0.0, 0.0, 1.0, ..., 0.0, 0.0, 0],
       ...,
       [1.0, 0.0, 0.0, ..., 0.0, 1.0, 3],
       [1.0, 0.0, 0.0, ..., 1.0, 0.0, 3],
       [1.0, 0.0, 0.0, ..., 0.0, 0.0, 3]], shape=(12960, 24), dtype=object)

In [6]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
target = encoder.fit_transform(target)

target

array([2, 1, 0, ..., 3, 3, 0], shape=(12960,))

In [7]:
from sklearn.model_selection import train_test_split
descriptive_train, descriptive_test, target_train, target_test = train_test_split(descriptive, target, test_size=0.2, random_state=0)

In [8]:
from sklearn.preprocessing import StandardScaler
standard_scaler = StandardScaler()

descriptive_train[:, :] = standard_scaler.fit_transform(descriptive_train[:, :])
descriptive_test[:, :] = standard_scaler.transform(descriptive_test[:, :])

descriptive_train, descriptive_test

(array([[1.424712999285516, -0.706646475106794, -0.7127922048344713, ...,
         -0.7100243864362177, 1.4145205218737622, 0.44926947563032077],
        [-0.7018957505837944, 1.4151347742147353, -0.7127922048344713,
         ..., 1.408402329699237, -0.7069533347423477,
         -0.44598824555652655],
        [1.424712999285516, -0.706646475106794, -0.7127922048344713, ...,
         1.408402329699237, -0.7069533347423477, 0.44926947563032077],
        ...,
        [1.424712999285516, -0.706646475106794, -0.7127922048344713, ...,
         -0.7100243864362177, -0.7069533347423477, 0.44926947563032077],
        [1.424712999285516, -0.706646475106794, -0.7127922048344713, ...,
         -0.7100243864362177, -0.7069533347423477, 1.3445271968171681],
        [-0.7018957505837944, -0.706646475106794, 1.402933412034638, ...,
         -0.7100243864362177, -0.7069533347423477, 0.44926947563032077]],
       shape=(10368, 24), dtype=object),
 array([[-0.7018957505837944, 1.4151347742147353, -0.7127

In [9]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)
classifier.fit(descriptive_train, target_train)

KNeighborsClassifier()

In [10]:
prediction = classifier.predict(descriptive_test)
prediction, target_test

(array([1, 3, 3, ..., 1, 0, 0], shape=(2592,)),
 array([1, 3, 3, ..., 3, 0, 0], shape=(2592,)))

In [11]:
from sklearn.metrics import confusion_matrix, accuracy_score
accuracy = accuracy_score(target_test, prediction)
matrix = confusion_matrix(target_test, prediction)

In [12]:
accuracy

0.9552469135802469

In [13]:
matrix

array([[882,   0,   0,   0],
       [  0, 779,  39,  15],
       [  0,  51, 769,   0],
       [  0,  11,   0,  46]])